In [1]:

import os
import sys
import time
import threading
import json


    
    #Creates a unique file name for datastore by appending epoch timestamp to the file name
    
def get_file_name():
    import uuid
    uniq_append_string = uuid.uuid4().hex
    return "LOCAL_STORAGE_{}".format(uniq_append_string)

    #Creates a unique file name with actual path for datastore and return 
    
def get_instance(file_name=None):
    if file_name is None:
        file_name = get_file_name()
    import os

    try:
        os.mkdir('C:/tmp')
    except:
        full_file_name = f"{'C:/tmp'}/{file_name+'.txt'}"
    return full_file_name



# Actual DataStore class 
class DataStore:

    # 'd' is the dictionary in which we store data
    def __init__(self, file_descriptor=None):
        self.__lock = threading.Lock()
        if(file_descriptor != None and os.path.isfile(file_descriptor)):
            self.file_descriptor = file_descriptor
        else:
            self.file_descriptor = get_instance()
        self.f = open(self.file_descriptor, 'w+')
        #
        self.d = dict(self.f.read())
    # for create operation
    # use syntax "create(key_name,value,timeout_value)" timeout is optional you can continue by passing two arguments without timeout

    def create(self, key, value, timeout=0):
        self.__lock.acquire()
        try:
            self.key = key
            self.value = value
            self.timeout = timeout
            if self.key in self.d:
                print("error: this key already exists")  # error message1
            else:
                if(self.key.isalpha()):
                    # constraints for file size less than 1GB and Jasonobject value less than 16KB
                    if (sys.getsizeof(self.d) < (1024*1024*1024) and sys.getsizeof(self.value) <= (16000)):
                        if (self.timeout == 0):
                            self.l = [self.value, self.timeout]
                        else:
                            self.l = [self.value, time.time()+self.timeout]

                        if (len(self.key) <= 32):  # constraints for input key_name capped at 32chars
                            self.d[self.key] = self.l
                        else:
                            print("error: Key Size must be between 32 character")
                            
                    else:
                        # error message2
                        print("error: Memory limit exceeded!! ")
                else:
                    # error message3
                    print(
                        "error: Invalind key_name!! key_name must contain only alphabets and no special characters or numbers")
        finally:
            self.__lock.release()

    # for read operation
    # use syntax "read(key_name)"

    def read(self, key):
        self.__lock.acquire()
        try:
            self.json = {}
            self.key = key
            if (self.key not in self.d):
                # error message4
                print(
                    "error: given key does not exist in database. Please enter a valid key")
            else:
                self.b = self.d[key]
                if (self.b[1] != 0):
                    # comparing the present time with expiry time
                    if (time.time() < self.b[1]):
                        # to return the value in the format of JasonObject i.e.,"key_name:value"
                        if(type(self.b[0]) != dict()):
                            res = dict(self.b[0])

                        return res
                    else:
                        print("error: time-to-live of", self.key,
                              "has expired")  # error message5
                else:
                    if(type(self.b[0]) != dict()):
                        res = dict(self.b[0])

                    return res
        finally:
            self.__lock.release()

    # for delete operation
    # use syntax "delete(key_name)"

    def delete(self, key):
        self.key = key
        if (self.key not in self.d):
            # error message4
            raise "error: given key does not exist in database. Please enter a valid key"
        else:
            self.b = self.d[key]
            if (self.b[1] != 0):
                # comparing the current time with expiry time
                if (time.time() < self.b[1]):
                    del self.d[key]
                    print("key is successfully deleted")
                else:
                    print("error: time-to-live of", self.key,
                          "has expired")  # error message5
            else:
                del self.d[key]
                print("key is successfully deleted")
    # it Will save the data in a logical file on path : C:/tmp/*.txt
    def save(self):
        x = json.dumps(self.d, indent=4)
        self.f.write(x)
        self.f.close()



In [2]:
#instantiateing The DataStore Class

datastore = DataStore()

### Create Key-Value datastore using create() method 

In [3]:
#to create a key with key_name,value given and with no time-to-live property
datastore.create('student',{'name':'amit','roll':14})

In [4]:
#to create a key with key_name,value given and with time-to-live property (120 second)
datastore.create('studenttwo',{'name':'polash','roll':20},120) 


In [5]:
#to create a key with key_name,value given and with time-to-live property (3600 second)
datastore.create("teacher",{'name':"Rafi",'age':30},3600)

### Access All the key which is presnt on Data Store using read() method

###### Access Value from Data store using read() method . It return JSON format output

In [6]:
# Access Value from Data store using read() method . It return JSON format output
datastore.read('student')

{'name': 'amit', 'roll': 14}

In [7]:
datastore.read('studenttwo')

{'name': 'polash', 'roll': 20}

In [8]:
datastore.read('teacher')

{'name': 'Rafi', 'age': 30}

### check time-to-live property
*** studenttwo key had 120sec time-to-live property ***


In [9]:
#After 120 sec 
# after 120 sec it will expaired
datastore.read('studenttwo')

error: time-to-live of studenttwo has expired


In [10]:
#After 120 sec
#student key has no time-to-live property 
datastore.read('student')

{'name': 'amit', 'roll': 14}

In [11]:
#student key has  time-to-live property (3600 sec) 
datastore.read('teacher')

{'name': 'Rafi', 'age': 30}

### Delete() method check 

In [12]:
datastore.delete('teacher')

key is successfully deleted


In [13]:
datastore.read('teacher')

error: given key does not exist in database. Please enter a valid key


In [14]:
datastore.delete('studenttwo')

error: time-to-live of studenttwo has expired


### key name have some Constraints also 
***Lets check it***

In [15]:
#key will only support 32 character of Alphabet
datastore.create('student123',{'name': 'polash', 'roll': 20})
# student123 this key contain integer in it so it gives error

error: Invalind key_name!! key_name must contain only alphabets and no special characters or numbers


In [16]:
#key will only support 32 character of Alphabet
datastore.create('student_djs',{'name': 'polash', 'roll': 20})
# student123 this key contain underscore(_) in it so it gives error

error: Invalind key_name!! key_name must contain only alphabets and no special characters or numbers


In [17]:
#key will only support 32 character of Alphabet
datastore.create('abcdsjdhfsjdhdfjdhfhdhfdhfhdfjskfhskfhsfhsdfhsdjfsdjkhfsdhfdfhdfsdkjfhskdfhsdhfsdkfh',{'name': 'polash', 'roll': 20})
# student123 this key contain integer in it so it gives error

error: Key Size must be between 32 character


### We can save this data in text file 
### save()
***This method will store the data in a unique file (C:/tmp/*.txt)***

In [18]:
datastore.save()

the code also returns other errors like <br>
#"invalidkey" if key_length is greater than 32 or key_name contains any numeric,special characters etc. <br>
#"key doesnot exist" if key_name was mis-spelt or deleted earlier<br>
#"File memory limit reached" if file memory exceeds 1GB<br>